In [6]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from collections import deque
import random
import gymnasium as gym
from gymnasium import spaces
import datetime
import my_library as mylib
import statsmodels.api as sm
import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None

Environment Definition

In [7]:
class TradingEnv(gym.Env):
    def __init__(self, df, rf):
        super(TradingEnv, self).__init__()

        self.current_step = 0
        self.data = df.iloc[:,df.columns != 'log_ret']
        self.y = df['log_ret']
        self.initial_position = pd.DataFrame({'0': [0], '1': [1]}) # Initial position is 1000 in risk_free asset
        self.position = self.initial_position
        self.initial_portfolio_value = list([1000])
        self.money = pd.DataFrame(self.initial_portfolio_value[0] * self.position)
        self.rf = rf
        self.pos_sup = 2
        self.pos_inf = -1
        # Define action and observation space
        
        # Only one stock, the rest is in cash
        self.action_space = np.arange(self.pos_inf, self.pos_sup, 0.15).round(2)
        # If two stocks, then self.action_space = spaces.Box(low=np.array([0,0]), high=np.array([1,1]), dtype=np.float32)
        
        # observation_space is a dataframe with historical data
        self.observation_space = spaces.Box(low=0, high=np.inf, shape=(1,), dtype=np.float32)
           
    def step(self, state, action):
        # Execute one time step within the environment
        self.current_step += 1
        self.flag = 0

        # End the episode if we've run out of data
        if self.current_step >= days_train:
            return self.position, 0, True

        # Take the action (<0: short stock, ==0: hold positions, >0: buy stock)
        # action = action[0][0]
        if (action > self.pos_sup) or (action < self.pos_inf):
            reward = - 1000
            self.flag = 1
        new_port_value = np.exp(state[0]) * self.money.iloc[self.current_step-1,0] + self.money.iloc[self.current_step-1,1] * (1+self.rf)
        self.portfolio_value.append(new_port_value)
        
        if abs(action - self.position.iloc[self.current_step-1,0]) >= tol:
            new_row_pos = pd.DataFrame([[action, (1 - action)]])
        else:
            new_row_pos = pd.DataFrame([[self.position.iloc[self.current_step-1,0], self.position.iloc[self.current_step-1,1]]])
        self.position = pd.concat([self.position, new_row_pos], axis=0, ignore_index=True)    
        new_row_money = pd.DataFrame([[self.position.iloc[self.current_step,0]*self.portfolio_value[self.current_step], self.position.iloc[self.current_step,1]*self.portfolio_value[self.current_step]]])
        self.money = pd.concat([self.money, new_row_money], axis=0, ignore_index=True)

        if self.flag == 0:
            reward = (self.portfolio_value[self.current_step] - self.portfolio_value[self.current_step-1])/self.portfolio_value[self.current_step-1]
        
        done = self.current_step >= days_train - 1
        
        observation = np.concatenate(([self.y_train.iloc[self.current_step]], self.df_train.iloc[self.current_step], [self.position.iloc[self.current_step,0], self.position.iloc[self.current_step,1], self.money.iloc[self.current_step,0], self.money.iloc[self.current_step,1], self.portfolio_value[self.current_step]]))

        return observation, reward, done

    def reset(self):
        # Reset the state of the environment to an initial state
        self.current_step = 0
        a = (3 * np.random.rand(1) - 1)[0].round(5)
        self.position = pd.DataFrame([a,1-a]).T
        self.portfolio_value = self.initial_portfolio_value
        self.money = pd.DataFrame(self.portfolio_value[0] * self.position)
        starting_point = np.random.randint(0, self.data.shape[0]-days_train)
        self.df_train = self.data.iloc[starting_point:starting_point+days_train,]
        self.y_train = self.y.iloc[starting_point:starting_point+days_train,]
        for i in range(days_train):
            b = (0.05 * np.random.rand(1) - 0.025)[0].round(5)
            self.y_train.iloc[i] *= (1+b)
            for j in range(len(self.df_train.columns)):
                c = (0.05 * np.random.rand(1) - 0.025)[0].round(5)
                self.df_train.iloc[i,j] *= (1+c)

        # The first observation is the returns of the first day, the initial position, and the initial portfolio value
        observation = np.insert(self.df_train.iloc[self.current_step].values.flatten(),0,self.y_train.iloc[self.current_step])
        observation = np.concatenate((observation, [self.position.iloc[self.current_step,0], self.position.iloc[self.current_step,1], self.money.iloc[self.current_step,0], self.money.iloc[self.current_step,1], self.portfolio_value[self.current_step]]))

        return observation

    def render(self, mode='human'):
        # Render the environment to the screen
        if mode == 'human':
            print(f"Current Step: {self.current_step}")
            # print(f"Current Price: {self.df_train.iloc[self.current_step]}")
            print(f"Current Position:\n Stock: {self.position.iloc[self.current_step,0]}\n R_f: {self.position.iloc[self.current_step,1]}")
            print(f"env.money = {self.money.iloc[self.current_step]}")
            print(f"Portfolio Value: {self.portfolio_value[self.current_step]}")
    
    def train(self, episodes, days):
        self.days_train = days
        for e in range(episodes):
            print(f"Episode: {e+1}/{episodes}")
            state = env.reset()
            # state = np.reshape(state, [1, state_size])
            for time in range(days): 
                action = agent.act(state)*0.15-1
                print(f"log_return: {state[0]}")
                next_state, reward, done = env.step(state,action)
                if agent.flag == 1:
                    agent.reward_pred *= (1+reward)
                    agent.flag = 0
                # next_state = np.reshape(next_state, [1, state_size])
                agent.remember(state, action, reward, next_state, done)
                self.render()
                print(f"Reward: {reward}\n")
                state = next_state
                if done:
                    print(f"Episode: {e+1}/{episodes}, Score: {time}, Epsilon: {agent.epsilon:.2}\n\n\n")
                    break
            if len(agent.memory) > 32:
                agent.replay(32)
        

Agent Definition

In [8]:
class DQN:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=3000)
        self.gamma = 0.95  # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()
        self.reward_pred = 1
        self.n_exploit = 0
        self.flag = 0

    def _build_model(self):
        model = Sequential()
        model.add(Dense(64, input_dim=self.state_size, activation='sigmoid'))
        model.add(Dense(64, activation='sigmoid'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(learning_rate=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        print("EXPLOITATION")
        self.flag = 1
        self.n_exploit += 1
        return np.argmax(act_values[0])

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = (reward + self.gamma * np.amax(self.model.predict(next_state)[0]))
            target_f = self.model.predict(state)
            target_f[0][int(action)] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

Global variables setting

In [2]:
rf = (1+0.02)**(1/252) - 1
tol = 0.01
episodes = 10
days_train = 100

Data manipulation

In [3]:
data = pd.read_csv('AAPL.csv', sep=';')
data['Date'] = pd.to_datetime(data['Date'],format='%d/%m/%Y')
adj_close = data['Adj Close']
df = pd.DataFrame({'Date': data['Date'], 'log_ret': np.log(adj_close/adj_close.shift(1)).dropna()}).dropna().set_index('Date')
sp500 = pd.read_csv('sp500_joined_closes.csv', sep=',')
data.set_index('Date', inplace=True)
new_cols = list([])
for i, col in enumerate(sp500.columns):
    if col.endswith(' '):
        new_cols.append(col[:-1])
    else:
        new_cols.append(col)
sp500.columns = new_cols
sp500['Date'] = pd.to_datetime(sp500['Date'])
sp500 = sp500.set_index('Date')
sp500.drop('AAPL', axis=1, inplace=True)

In [4]:
sp500_lr = np.log(sp500/sp500.shift(1))
sp500_lr = sp500_lr.drop(sp500_lr.index[0])
sp500_lr = sp500_lr.drop([col for col in sp500_lr.columns if sp500_lr[col].count() < 3633], axis=1)
df = df.join(sp500_lr,how='inner').dropna()

Performing PCA on AAPL and SP500 stocks

In [11]:
cum_var_exp, PCs, regr_PCs = mylib.PCA(df['log_ret'], sp500_lr, plot=False)

In [13]:
regr_PCs.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   AAPL   R-squared:                       0.464
Model:                            OLS   Adj. R-squared:                  0.463
Method:                 Least Squares   F-statistic:                     314.1
Date:                Sun, 07 Jul 2024   Prob (F-statistic):               0.00
Time:                        13:16:34   Log-Likelihood:                -4020.1
No. Observations:                3633   AIC:                             8062.
Df Residuals:                    3622   BIC:                             8130.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       2.395e-17      0.012   1.97e-15      1.000      -0.024       0.024
PC0            0.0456      0.001     48.614      0.000       0.044       0.047
PC1            0.0156      0.003      5.676      0.000       0.010       0.021
PC2           -0.0751      0.003    -22.826      0.000      -0.082      -0.069
PC3            0.0145      0.004      3.290      0.001       0.006       0.023
PC4            0.0389      0.005      7.646      0.000       0.029       0.049
PC5           -0.0080      0.006     -1.451      0.147      -0.019       0.003
PC6           -0.0074      0.006     -1.266      0.206      -0.019       0.004
PC7           -0.0679      0.006    -11.232      0.000      -0.080      -0.056
PC8           -0.0212      0.006     -3.314      0.001      -0.034      -0.009
PC9            0.0253      0.007      3.841      0.000       0.012       0.038
==============================================================================
Omnibus:                      573.429   Durbin-Watson:                   1.928
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            10155.212
Skew:                          -0.061   Prob(JB):                         0.00
Kurtosis:                      11.190   Cond. No.                         13.0
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [37]:
Y = PCs['AAPL']

In [23]:
[i-1 for i,p in enumerate(regr_PCs.tvalues) if abs(p) > 1.96]

[0, 1, 2, 3, 4, 7, 8, 9]

In [25]:
t_vals = regr_PCs.tvalues.drop('const')

In [36]:
t_vals.drop([p for p in t_vals.index if abs(t_vals[p]) == abs(t_vals).min()], inplace=True)
list(t_vals.index)

['PC0', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC7', 'PC8', 'PC9']

In [39]:
X = PCs[list(t_vals.index)]


In [40]:
regr_PCs = sm.OLS(Y, X).fit()
regr_PCs.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                   AAPL   R-squared (uncentered):                   0.464
Model:                            OLS   Adj. R-squared (uncentered):              0.463
Method:                 Least Squares   F-statistic:                              348.9
Date:                Sun, 07 Jul 2024   Prob (F-statistic):                        0.00
Time:                        13:49:08   Log-Likelihood:                         -4020.9
No. Observations:                3633   AIC:                                      8060.
Df Residuals:                    3624   BIC:                                      8116.
Df Model:                           9                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
PC0            0.0456      0.001     48.617      0.000       0.044       0.047
PC1            0.0156      0.003      5.677      0.000       0.010       0.021
PC2           -0.0751      0.003    -22.828      0.000      -0.082      -0.069
PC3            0.0145      0.004      3.290      0.001       0.006       0.023
PC4            0.0389      0.005      7.647      0.000       0.029       0.049
PC5           -0.0080      0.006     -1.451      0.147      -0.019       0.003
PC7           -0.0679      0.006    -11.232      0.000      -0.080      -0.056
PC8           -0.0212      0.006     -3.314      0.001      -0.034      -0.009
PC9            0.0253      0.007      3.841      0.000       0.012       0.038
==============================================================================
Omnibus:                      571.772   Durbin-Watson:                   1.928
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            10041.479
Skew:                          -0.066   Prob(JB):                         0.00
Kurtosis:                      11.144   Cond. No.                         7.02
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [47]:
t_vals = regr_PCs.tvalues
t_vals.drop([p for p in t_vals.index if abs(t_vals[p]) == abs(t_vals).min()], inplace=True)
list(t_vals.index)
X = PCs[list(t_vals.index)]
X = sm.add_constant(X)
regr_PCs = sm.OLS(Y, X).fit()
regr_PCs.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   AAPL   R-squared:                       0.464
Model:                            OLS   Adj. R-squared:                  0.463
Method:                 Least Squares   F-statistic:                     392.0
Date:                Sun, 07 Jul 2024   Prob (F-statistic):               0.00
Time:                        15:28:55   Log-Likelihood:                -4021.9
No. Observations:                3633   AIC:                             8062.
Df Residuals:                    3624   BIC:                             8118.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       2.395e-17      0.012   1.97e-15      1.000      -0.024       0.024
PC0            0.0456      0.001     48.603      0.000       0.044       0.047
PC1            0.0156      0.003      5.675      0.000       0.010       0.021
PC2           -0.0751      0.003    -22.821      0.000      -0.082      -0.069
PC3            0.0145      0.004      3.289      0.001       0.006       0.023
PC4            0.0389      0.005      7.644      0.000       0.029       0.049
PC7           -0.0679      0.006    -11.229      0.000      -0.080      -0.056
PC8           -0.0212      0.006     -3.314      0.001      -0.034      -0.009
PC9            0.0253      0.007      3.840      0.000       0.012       0.038
==============================================================================
Omnibus:                      571.083   Durbin-Watson:                   1.931
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             9984.148
Skew:                          -0.070   Prob(JB):                         0.00
Kurtosis:                      11.120   Cond. No.                         13.0
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [48]:
t_vals = regr_PCs.tvalues
any(abs(t_vals) < 1.96)

True

In [50]:
t_vals = regr_PCs.tvalues
t_vals.drop([p for p in t_vals.index if abs(t_vals[p]) == abs(t_vals).min()], inplace=True)
list(t_vals.index)
X = PCs[list(t_vals.index)]
regr_PCs = sm.OLS(Y, X).fit()
regr_PCs.summary()
t_vals = regr_PCs.tvalues
any(abs(t_vals) < 1.96)

False

In [54]:
regr_PCs.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                   AAPL   R-squared (uncentered):                   0.464
Model:                            OLS   Adj. R-squared (uncentered):              0.463
Method:                 Least Squares   F-statistic:                              392.1
Date:                Sun, 07 Jul 2024   Prob (F-statistic):                        0.00
Time:                        15:30:54   Log-Likelihood:                         -4021.9
No. Observations:                3633   AIC:                                      8060.
Df Residuals:                    3625   BIC:                                      8109.
Df Model:                           8                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
PC0            0.0456      0.001     48.610      0.000       0.044       0.047
PC1            0.0156      0.003      5.676      0.000       0.010       0.021
PC2           -0.0751      0.003    -22.824      0.000      -0.082      -0.069
PC3            0.0145      0.004      3.290      0.001       0.006       0.023
PC4            0.0389      0.005      7.646      0.000       0.029       0.049
PC7           -0.0679      0.006    -11.230      0.000      -0.080      -0.056
PC8           -0.0212      0.006     -3.314      0.001      -0.034      -0.009
PC9            0.0253      0.007      3.840      0.000       0.012       0.038
==============================================================================
Omnibus:                      571.083   Durbin-Watson:                   1.931
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             9984.148
Skew:                          -0.070   Prob(JB):                         0.00
Kurtosis:                      11.120   Cond. No.                         7.02
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Add the relevant PCs to the df

In [57]:
PCs['AAPL']

Date
2010-01-05    0.044287
2010-01-06   -0.960478
2010-01-07   -0.158106
2010-01-08    0.321426
2010-01-11   -0.554678
                ...   
2024-06-05    0.387274
2024-06-06   -0.456310
2024-06-07    0.643276
2024-06-10   -1.147117
2024-06-11    1.867397
Name: AAPL, Length: 3633, dtype: float64

In [56]:
dataframe = pd.DataFrame(PCs[list(t_vals.index)])
dataframe['log_ret'] = df['log_ret']
dataframe

,PC0,PC1,PC2,PC3,PC4,PC7,PC8,PC9,log_ret
Date,,,,,,,,,
2010-01-05,2.277725,5.598344,3.573229,1.501416,-2.727350,-0.331074,-3.486098,0.800556,0.001727
2010-01-06,1.487877,0.917696,1.890682,-0.739696,2.193443,2.344322,2.210504,-0.477584,-0.016034
2010-01-07,5.457933,3.292415,3.593552,1.434769,-6.515021,2.830913,1.448740,-0.823186,-0.001850
2010-01-08,2.347663,3.042308,-1.758965,-1.573010,3.061445,0.025891,2.377851,-1.518725,0.006626
2010-01-11,3.132203,-3.110919,1.425089,0.095026,0.606712,0.379562,0.458655,-3.778597,-0.008861
...,...,...,...,...,...,...,...,...,...
2024-06-05,4.370851,7.734809,-5.675138,2.531673,0.669577,0.257385,0.070731,-0.569867,0.007790
2024-06-06,-3.327944,-0.154414,-0.253791,-1.452974,0.201252,0.250025,-2.509249,2.491141,-0.007122
2024-06-07,-3.678136,3.166701,1.440477,-2.540191,-2.242545,-1.479271,0.525229,-0.008270,0.012316


Create the environment and agent, training

In [30]:
env = TradingEnv(dataframe, rf)
state_size = env.observation_space.shape[0]
action_size = len(env.action_space)
agent = DQN(state_size, action_size)

env.train(episodes, days_train)
print(f"Annualized rewards predicted: {(agent.reward_pred**(252/agent.n_exploit)-1)*100:.2f}%")


C:\Users\pietr\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Episode: 1/10
log_return: -0.03836061767234994
Current Step: 1
Current Position:
 Stock: 0.6499999999999999
 R_f: 0.3500000000000001
env.money = 0    631.802162
1    340.201164
Name: 1, dtype: float64
Portfolio Value: 972.0033261496974
Reward: -0.027996673850302613

log_return: 0.028366424369132944
Current Step: 2
Current Position:
 Stock: -0.85
 R_f: 1.85
env.money = 0    -841.677345
1    1831.885985
Name: 2, dtype: float64
Portfolio Value: 990.2086408005221
Reward: 0.018729683490837038

log_return: -0.006529010969374214
Current Step: 3
Current Position:
 Stock: 0.6499999999999999
 R_f: 0.3500000000000001
env.money = 0    647.289513
1    348.540507
Name: 3, dtype: float64
Portfolio Value: 995.8300195449775
Reward: 0.0056769639375302975

log_return: 0.0063387150489206835
Current Step: 4
Current Position:
 Stock: 1.1
 R_f: -0.10000000000000009
env.money = 0    1099.970767
1     -99.997342
Name: 4, dtype: float64
Portfolio Value: 999.9734246976118
Reward: 0.0041607554214197215

log_retur